## import libraries

In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim
import requests


In [8]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [9]:
df_toronto = pd.read_csv(r'C:\Users\w5080797\Documents\Personal\IBM_qualification\df_neighborhoods_coordinates.csv')
df_toronto.head()

,Unnamed: 0,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,0,Downtown Toronto,Rosedale,M4W,43.679563,-79.377529
1,1,East York,East Toronto,M4J,43.685347,-79.338106
2,2,North York,Henry Farm,M2J,43.778517,-79.346556
3,3,North York,Hillcrest Village,M2H,43.803762,-79.363452
4,4,Scarborough,Woburn,M1G,43.770992,-79.216917


## create a map of toronto

In [10]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

## create a new dataferame for Scarborough

In [27]:
#@hiddel_cell
CLIENT_ID = 'OVNJKI2111SKDXHSVBMPEJ1YKQYJ2KHJ4QUXKYGIEKKUFXDA'
CLIENT_SECRET = 'LPMXE1UZXFZ5GBEMWI2BXDZYHJFCKZOHSM5TFDKQCZJAC1QX'
VERSION = '20190416'

In [28]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Unnamed: 0,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,4,Scarborough,Woburn,M1G,43.770992,-79.216917
1,8,Scarborough,"Cliffcrest, Cliffside",M1M,43.716316,-79.239476
2,21,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
3,23,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
4,24,Scarborough,Tam O'Shanter,M1T,43.781638,-79.304302
5,25,Scarborough,"Morningside, West Hill",M1E,43.763573,-79.188711
6,31,Scarborough,Upper Rouge,M1X,43.836125,-79.205636


## create a map of Scarborough

In [29]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [30]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

## top 100 venues in the neighbourhood Steeles West in Scarborough

In [31]:

neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of "Woburn" are 43.7709921, -79.21691740000001.


In [32]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)


In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cb5a38d1ed2196d925576c5'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5085ec39e4b0b1ead2eb0818-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/toys_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1f3941735',
         'name': 'Toy / Game Store',
         'pluralName': 'Toy / Game Stores',
         'primary': True,
         'shortName': 'Toys & Games'}],
       'id': '5085ec39e4b0b1ead2eb0818',
       'location': {'address': '300 Borough Drive',
        'cc': 'CA',
        'city': 'Scarborough',
        'country': 'Canada',
        'crossStreet': 'in Scarborough Town Centre',
        'distance': 284,
        'formattedAddress': ['300 Borough Drive (in Scarborough Town Centre)',
         'Scarborough ON M1P 4P5

In [34]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [35]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,DAVIDsTEA,Tea Room,43.776613,-79.258516
5,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
6,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
7,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
8,Jimmy The Greek,Greek Restaurant,43.775112,-79.257119
9,Scarborough Town Centre,Shopping Mall,43.775237,-79.257363


In [36]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

43 venues were returned by Foursquare.


In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Woburn
Cliffcrest, Cliffside
Scarborough Village
Rouge, Malvern
Tam O'Shanter
Morningside, West Hill
Upper Rouge
Agincourt
Highland Creek, Rouge Hill, Port Union
Dorset Park, Scarborough Town Centre, Wexford Heights
Agincourt North, Milliken
Birch Cliff
Clairlea, Golden Mile, Oakridge
Maryvale, Wexford
Ionview, Kennedy Park


In [39]:
scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
1,Woburn,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop
2,Woburn,43.770992,-79.216917,Korean Grill House,43.770812,-79.214502,Korean Restaurant
3,"Cliffcrest, Cliffside",43.716316,-79.239476,Have A Nap Motel,43.718256,-79.240135,Motel
4,"Cliffcrest, Cliffside",43.716316,-79.239476,Vincent's Spot,43.717002,-79.242353,American Restaurant
5,"Cliffcrest, Cliffside",43.716316,-79.239476,Avon Motel,43.720006,-79.239164,Motel
6,Scarborough Village,43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground
7,Scarborough Village,43.744734,-79.239476,Tumbe Cafe,43.744058,-79.244021,Grocery Store
8,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
9,Tam O'Shanter,43.781638,-79.304302,Remezzo Italian Bistro,43.778649,-79.308264,Italian Restaurant


In [40]:
scarborough_venues.tail(10)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
56,"Maryvale, Wexford",43.750071,-79.295849,Lebanese bakery,43.746701,-79.292896,Middle Eastern Restaurant
57,"Maryvale, Wexford",43.750071,-79.295849,Wexford Heights Plaza,43.746136,-79.293782,Shopping Mall
58,"Maryvale, Wexford",43.750071,-79.295849,Scarborough Garage Door Repair,43.751288,-79.301508,Auto Garage
59,"Maryvale, Wexford",43.750071,-79.295849,Sequoia Lounge,43.745645,-79.295737,Middle Eastern Restaurant
60,"Maryvale, Wexford",43.750071,-79.295849,Crown Pastries,43.746098,-79.293142,Bakery
61,"Ionview, Kennedy Park",43.727929,-79.262029,Giant Tiger,43.727447,-79.266240,Department Store
62,"Ionview, Kennedy Park",43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,Coffee Shop
63,"Ionview, Kennedy Park",43.727929,-79.262029,Bros. CONVENIENCE,43.727781,-79.265708,Convenience Store
64,"Ionview, Kennedy Park",43.727929,-79.262029,Tandy Leather,43.726974,-79.266513,Hobby Shop
65,"Ionview, Kennedy Park",43.727929,-79.262029,Dollarama,43.726596,-79.266830,Discount Store


In [41]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",2,2,2,2,2,2
Birch Cliff,4,4,4,4,4,4
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Cliffcrest, Cliffside",3,3,3,3,3,3
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"Highland Creek, Rouge Hill, Port Union",1,1,1,1,1,1
"Ionview, Kennedy Park",5,5,5,5,5,5
"Maryvale, Wexford",7,7,7,7,7,7


In [42]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))


There are 46 uniques categories.


In [43]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Auto Garage,Bakery,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,Woburn,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Woburn,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Woburn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Cliffcrest, Cliffside",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Cliffcrest, Cliffside",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
scarb_onehot.shape

(66, 47)

In [45]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,American Restaurant,Auto Garage,Bakery,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.0,0.0,0.0,0.25,0.0,0.0,0.00,0.000000,0.25,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000
1,"Agincourt North, Milliken",0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.5,0.000000,0.0,0.0,0.5,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000
2,Birch Cliff,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.25,0.000000,0.00,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000
3,"Clairlea, Golden Mile, Oakridge",0.000000,0.0,0.2,0.0,0.00,0.2,0.1,0.00,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.000000,0.1,0.0,0.0,0.000000,0.00,0.0,0.1,0.0,0.0,0.000000,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.1,0.0,0.0,0.000000
4,"Cliffcrest, Cliffside",0.333333,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000
5,"Dorset Park, Scarborough Town Centre, Wexford ...",0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.166667,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.333333,0.0,0.0,0.0,0.166667,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.166667
6,"Highland Creek, Rouge Hill, Port Union",0.000000,0.0,0.0,1.0,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000


## top 10 venues per neighbourhood

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North, Milliken",Park,Playground,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,Birch Cliff,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
3,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Park,Soccer Field,Breakfast Spot,Department Store
4,"Cliffcrest, Cliffside",Motel,American Restaurant,College Stadium,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
5,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Pet Store,Convenience Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
6,"Highland Creek, Rouge Hill, Port Union",Bar,Vietnamese Restaurant,College Stadium,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
7,"Ionview, Kennedy Park",Coffee Shop,Hobby Shop,Discount Store,Department Store,Convenience Store,College Stadium,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
8,"Maryvale, Wexford",Middle Eastern Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
9,"Morningside, West Hill",Intersection,Spa,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,College Stadium,Fried Chicken Joint


## k mean s clustering to group the neignbourhodds into 5 clusters

In [54]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(14)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([0, 1, 0, 0, 3, 0, 4, 0, 0, 0], dtype=int32)

## add k means labels into the original dataframe

In [55]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,Unnamed: 0,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,Scarborough,Woburn,M1G,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,College Stadium,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
1,8,Scarborough,"Cliffcrest, Cliffside",M1M,43.716316,-79.239476,1,Motel,American Restaurant,College Stadium,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,21,Scarborough,Scarborough Village,M1J,43.744734,-79.239476,0,Grocery Store,Playground,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
3,23,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,College Stadium,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
4,24,Scarborough,Tam O'Shanter,M1T,43.781638,-79.304302,3,Pizza Place,Noodle House,Fried Chicken Joint,Thai Restaurant,Italian Restaurant,Shopping Mall,Fast Food Restaurant,Pharmacy,Chinese Restaurant,Coffee Shop
5,25,Scarborough,"Morningside, West Hill",M1E,43.763573,-79.188711,0,Intersection,Spa,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,College Stadium,Fried Chicken Joint
6,31,Scarborough,Upper Rouge,M1X,43.836125,-79.205636,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,36,Scarborough,Agincourt,M1S,43.794200,-79.262029,0,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
8,47,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497,0,Bar,Vietnamese Restaurant,College Stadium,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,48,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P,43.757410,-79.273304,0,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Pet Store,Convenience Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


## visualise the clusters

In [56]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
check each of the clusters